<a href="https://colab.research.google.com/github/HromovAndrey/II_mood_4_part_4/blob/main/%D0%A3%D1%80%D0%BE%D0%BA_%D0%BC%D0%BE%D0%B4%D1%83%D0%BB%D1%8C_4_%D1%87%D0%B0%D1%81%D1%82%D1%8C_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import torch
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_device(device)

In [29]:

df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module3/data/Airline%20Passenger%20Satisfaction.csv",
                 usecols=['Ease of Online booking', 'Inflight wifi service', 'Food and drink', 'Seat comfort', 'satisfaction']
                 )

df['satisfaction'] = df['satisfaction'].map({'satisfied': 1, 'neutral or dissatisfied': 0})

df.head()

,Inflight wifi service,Ease of Online booking,Food and drink,Seat comfort,satisfaction
0,5,3,3,3,1
1,1,3,5,5,1
2,2,2,2,2,0
3,0,0,3,4,1
4,2,4,4,2,1


In [30]:
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

class MyDataset(Dataset):
  def __init__(self, X, y):
    self.X = torch.FloatTensor(X.values)
    self.y = torch.FloatTensor(y.values)
    self.y = self.y.reshape(-1, 1)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]

X = df.drop(columns=['satisfaction'])
y = df['satisfaction']
dataset = MyDataset(X, y)

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.8,
                                                    random_state=42,
                                                    stratify=y)
dataset_train = MyDataset(X_train, y_train)
dataset_test = MyDataset(X_test, y_test)

In [32]:
from torch.utils.data import DataLoader

batch_size = 512
train_dl = DataLoader(dataset_test,
                      batch_size=batch_size,
                      shuffle=True)



In [33]:
import numpy as np
from torch import nn
import torch.nn.functional as F

class Block(nn.Module):
  def __init__(self, input_dim, output_dim):
    super().__input__()

    self.linear = nn.Linear(input_dim, output_dim)
    self.linear1 = nn.Linear(output_dim, output_dim)
    self.linear2 = nn.Linear(output_dim, output_dim)

    self.bn1 = nn.BatchNormald(output_dim)
    self.bn2 = nn.BatchNormald(output_dim)

  def forward(self, x):
    x = self.linear(x)
    x = F.relu(x)

    out = self.linear1(x)
    out = self.bn1(out)
    out = F.relu(out)

    return out + x



In [34]:
class BinaryClassifier(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super().__init__()

    self.all_layers = nn.Sequential(
        Block(input_dim, hidden_dim),
        nn.Dropout(0.2),
        Block(hidden_dim, hidden_dim//2),

        nn.Linear(hidden_dim//2, hidden_dim//2),
        nn.ReLU(),
        nn.Linear(hidden_dim//2, output_dim),
        nn.Sigmoid()
        )
    def forward(self, x):
      return self.all_layers(x)


    def predict(self, X, device='cpu'):
        X = torch.FloatTensor(np.array(X)).to(device)

        with torch.no_grad():
          y_pred = self.forward(X)

        return y_pred.cpu().numpy()


model = BinaryClassifier(4, 128, 1).to(device)
model


AttributeError: 'super' object has no attribute '__input__'